In [1]:
import json
import pandas as pd
import numpy as np
from dfply import *
import altair.vega.v5 as alt


In [2]:
# Prepare the GraphQL client
import requests
from IPython.display import display, Markdown
from gql import gql, Client
from gql.transport.requests import RequestsHTTPTransport

_transport = RequestsHTTPTransport(
    url='https://api.datacite.org/graphql',
    use_json=True,
)

client = Client(
    transport=_transport,
    fetch_schema_from_transport=True,
)

In [3]:
# Generate the GraphQL query to retrieve up to 100 outputs of University of Oxford, with at least 100 views each.
query_params = {
    "rorId" : "https://ror.org/052gg0110",
    "maxOutputs": 100,
    "minViews" : 100
}

query = gql("""query getOutputs($rorId: ID!, $maxOutputs: Int!, $minViews: Int!)
{
 organization(id: $rorId) {
    id
    name
    alternateName
    citationCount
    viewCount
    downloadCount
    works(hasViews: $minViews, first: $maxOutputs) {
      totalCount
      published {
        title
        count
      }
      resourceTypes {
        title
        count
      }
      nodes {
        id
        type
        publisher
        publicationYear
        titles {
          title
        }
        citations {
           nodes {
             id
             titles {
                title
             }
           }
        }
        creators {
          id
          name
          affiliation {
            id
            name
          }
        }
        citationCount
        viewCount
        downloadCount
      }
    }
  }
}
""")

In [4]:
data = client.execute(query, variable_values=json.dumps(query_params))

In [5]:
def get_data():
    response_string = '{"data":{"work":{"relatedIdentifiers":[{"relatedIdentifier":"10.5281\/zenodo.2648713","relatedIdentifierType":"DOI","relationType":"Cites"},{"relatedIdentifier":"10.5281\/zenodo.2648740","relatedIdentifierType":"DOI","relationType":"Cites"},{"relatedIdentifier":"10.5281\/zenodo.2648800","relatedIdentifierType":"DOI","relationType":"IsSupplementTo"},{"relatedIdentifier":"10.5281\/zenodo.2648751","relatedIdentifierType":"DOI","relationType":"HasVersion"}],"creators":[{"id":"https:\/\/orcid.org\/0000-0002-4537-6648","name":"Leidinger, Moritz","affiliation":[{"id":null}]}],"publisher":"Zenodo","identifiers":[{"identifier":"https:\/\/zenodo.org\/record\/2648751","identifierType":"URL"}],"fundingReferences":null,"repository":{"id":"cern.zenodo","re3dataId":"10.17616\/R3QP53","repositoryType":["other"]},"citations":{"totalCount":0,"published":[],"nodes":[]},"references":{"published":[],"nodes":[{ "id": "https://doi.org/10.15468/dl.ojaesd", "formattedCitation": "Occdownload Gbif.Org. (2015). <i>GBIF Occurrence Download</i> [Data set]. The Global Biodiversity Information Facility. https://doi.org/10.15468/DL.OJAESD</a>", "repository": { "name": "Global Biodiversity Information Facility", "re3dataId": "10.17616/R3J014", "id": "gbif.gbif" }, "registrationAgency": { "name": null, "id": null }, "member": { "name": "Global Biodiversity Information Facility", "id": "gbif" } }, { "id": "https://doi.org/10.15468/dl.uq8ndo", "formattedCitation": "Occdownload Gbif.Org. (2018). <i>GBIF Occurrence Download</i> [Data set]. The Global Biodiversity Information Facility. https://doi.org/10.15468/DL.UQ8NDO</a>", "repository": { "name": "Global Biodiversity Information Facility", "re3dataId": "10.17616/R3J014", "id": "gbif.gbif" }, "registrationAgency": { "name": "DataCite", "id": "datacite" }, "member": { "name": "Global Biodiversity Information Facility", "id": "gbif" } }]}}}}'
    return json.loads(response_string)["data"]["work"]

In [6]:
def add_node_attributes(dataframe, parent=2):
    return (dataframe >>
    mutate(
    id = X.id,
    parent = parent,
    ))
  

In [7]:
def create_node(array, parent=2):
    df = add_node_attributes(pd.DataFrame(array,columns=array[1].keys()), parent)
    return df.to_dict(orient='records')
     

In [8]:
def vega_template(data):
    return """
{
  "$schema": "https://vega.github.io/schema/vega/v3.json",
  "description": "An example of a radial layout for a node-link diagram of hierarchical data.",
  "width": 720,
  "height": 720,
  "padding": 5,
  "autosize": "none",
  "signals": [
    {"name": "labels", "value": true, "bind": {"input": "checkbox"}},
    {
      "name": "radius",
      "value": 280,
      "bind": {"input": "range", "min": 20, "max": 600}
    },
    {
      "name": "extent",
      "value": 360,
      "bind": {"input": "range", "min": 0, "max": 360, "step": 1}
    },
    {
      "name": "rotate",
      "value": 0,
      "bind": {"input": "range", "min": 0, "max": 360, "step": 1}
    },
    {
      "name": "layout",
      "value": "tidy",
      "bind": {"input": "radio", "options": ["tidy", "cluster"]}
    },
    {
      "name": "links",
      "value": "diagonal",
      "bind": {
        "input": "select",
        "options": ["line", "curve", "diagonal", "orthogonal"]
      }
    },
    {"name": "originX", "update": "width / 2"},
    {"name": "originY", "update": "height / 2"}
  ],
  "data": [
    {
      "name": "tree",
      "url": """ + data + """,
      "transform": [
        {"type": "stratify", "key": "id", "parentKey": "parent"},
        {
          "type": "tree",
          "method": {"signal": "layout"},
          "size": [1, {"signal": "radius"}],
          "as": ["alpha", "radius", "depth", "children"]
        },
        {
          "type": "formula",
          "expr": "(rotate + extent * datum.alpha + 270) % 360",
          "as": "angle"
        },
        {"type": "formula", "expr": "PI * datum.angle / 180", "as": "radians"},
        {
          "type": "formula",
          "expr": "inrange(datum.angle, [90, 270])",
          "as": "leftside"
        },
        {
          "type": "formula",
          "expr": "originX + datum.radius * cos(datum.radians)",
          "as": "x"
        },
        {
          "type": "formula",
          "expr": "originY + datum.radius * sin(datum.radians)",
          "as": "y"
        }
      ]
    },
    {
      "name": "links",
      "source": "tree",
      "transform": [
        {"type": "treelinks"},
        {
          "type": "linkpath",
          "shape": {"signal": "links"},
          "orient": "radial",
          "sourceX": "source.radians",
          "sourceY": "source.radius",
          "targetX": "target.radians",
          "targetY": "target.radius"
        }
      ]
    }
  ],
  "scales": [
    {
      "name": "color",
      "type": "linear",
      "range": {"scheme": "viridis"},
      "domain": {"data": "tree", "field": "depth"},
      "zero": true
    }
  ],
  "marks": [
    {
      "type": "path",
      "from": {"data": "links"},
      "encode": {
        "update": {
          "x": {"signal": "originX"},
          "y": {"signal": "originY"},
          "path": {"field": "path"},
          "stroke": {"value": "#ccc"}
        }
      }
    },
    {
      "type": "symbol",
      "from": {"data": "tree"},
      "encode": {
        "enter": {"size": {"value": 300}, "stroke": {"value": "#fff"}},
        "update": {
          "x": {"field": "x"},
          "y": {"field": "y"},
          "fill": {"scale": "color", "field": "depth"}
        }
      }
    },
    {
      "type": "text",
      "from": {"data": "tree"},
      "encode": {
        "enter": {
          "text": {"field": "name"},
          "fontSize": {"value": 12},
          "baseline": {"value": "middle"}
        },
        "update": {
          "x": {"field": "x"},
          "y": {"field": "y"},
          "dx": {"signal": "(datum.leftside ? -1 : 1) * 12"},
          "align": {"signal": "datum.leftside ? 'right' : 'left'"},
          "opacity": {"signal": "labels ? 1 : 0"}
        }
      }
    }
  ]
}

"""

In [9]:
def merge_nodes(dataset=[],funders=[],orgs=[],people=[]):
    dmp = {"id":1, "name": "dmp"}
    datasets_node = {"id":2, "name": "Datasets", "parent":1}
    funders_node = {"id":3, "name": "Funders", "parent":1}
    organisations_node = {"id":4, "name": "Organisations", "parent":1}
    people_node = {"id":5, "name": "People", "parent":1}
    nodes_list = [dmp, datasets_node, funders_node,organisations_node,people_node] + dataset + funders + orgs + people
    return np.array(nodes_list, dtype=object)

In [10]:
merge_nodes(create_node(get_data()["references"]["nodes"]))

array([{'id': 1, 'name': 'dmp'},
       {'id': 2, 'name': 'Datasets', 'parent': 1},
       {'id': 3, 'name': 'Funders', 'parent': 1},
       {'id': 4, 'name': 'Organisations', 'parent': 1},
       {'id': 5, 'name': 'People', 'parent': 1},
       {'id': 'https://doi.org/10.15468/dl.ojaesd', 'formattedCitation': 'Occdownload Gbif.Org. (2015). <i>GBIF Occurrence Download</i> [Data set]. The Global Biodiversity Information Facility. https://doi.org/10.15468/DL.OJAESD</a>', 'repository': {'name': 'Global Biodiversity Information Facility', 're3dataId': '10.17616/R3J014', 'id': 'gbif.gbif'}, 'registrationAgency': {'name': None, 'id': None}, 'member': {'name': 'Global Biodiversity Information Facility', 'id': 'gbif'}, 'parent': 2},
       {'id': 'https://doi.org/10.15468/dl.uq8ndo', 'formattedCitation': 'Occdownload Gbif.Org. (2018). <i>GBIF Occurrence Download</i> [Data set]. The Global Biodiversity Information Facility. https://doi.org/10.15468/DL.UQ8NDO</a>', 'repository': {'name': 'Global

## Create Node Structure



In [11]:
alt.vega(json.loads(vega_template('"https://gist.githubusercontent.com/kjgarza/f348f7566c6ff7950ffed8f5ae8d10a6/raw/ce0fff999a9c660a3f0e4c39fc3c19b7636e59ac/flare.json"')))

<Vega 5 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/troubleshooting.html
